<a href="https://colab.research.google.com/github/AlexanderPushkarev/LangChain/blob/main/%D0%A0%D0%9536_%22%D0%97%D0%B5%D1%80%D0%BE%D0%BA%D0%BE%D0%B4%D0%B5%D1%80_%D1%83%D1%80%D0%BE%D0%BA_%D0%BF%D0%BE_FAISS_(%D0%98_%D0%91%D0%BE%D0%B3%D0%B4%D0%B0%D0%BD)_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подгрузка библиотек и работа с базой данных


In [ ]:
#Устанавливаем библиотеки
!pip install -q langchain langchain-community sentence_transformers faiss-cpu
#если есть возможность ставить видеокарту, вместо последнего пишем faiss-gpu
#Чтобы работало без сбоев, можно зафиксировать версии, например langchain==0.2.6 sentence_transformers==3.0.1 faiss-cpu==1.7.4 langchain-community==0.2.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.6 MB/s eta 0:00:00


In [ ]:
!pip show langchain-community
#посмотреть версию какой-то из библиотек

Name: langchain-community
Version: 0.2.6
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
#Погрузка нужны для работы модулей

from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import re
import pandas as pd


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name='cointegrated/rubert-tiny2',
                                        # multi_process=True, model_kwargs={"device": "cuda"},
                                         encode_kwargs={"normalize_embeddings": True})
# есть работаем без виде карты удаляем или закомментируем кусок кода выше model_kwargs={"device": "cuda"}, иначе если не найдет видеокарту, выдаст ошибку. Без видеокарты будет работать медленнее


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Тестовая база на 20 ГБ
# ! wget 'https://opendata.trudvsem.ru/csv/cv.csv'

--2024-10-14 13:18:29--  https://opendata.trudvsem.ru/csv/cv.csv
Resolving opendata.trudvsem.ru (opendata.trudvsem.ru)... 185.99.9.49
Connecting to opendata.trudvsem.ru (opendata.trudvsem.ru)|185.99.9.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24069701172 (22G) [application/octet-stream]
Saving to: ‘cv.csv’

cv.csv               58%[==========>         ]  13.21G   101KB/s    in 33m 51s 

2024-10-14 13:52:23 (6.66 MB/s) - Read error at byte 14186055186/24069701172 (Connection reset by peer). Retrying.

--2024-10-14 13:52:24--  (try: 2)  https://opendata.trudvsem.ru/csv/cv.csv
Connecting to opendata.trudvsem.ru (opendata.trudvsem.ru)|185.99.9.49|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 24069701172 (22G), 9883645986 (9.2G) remaining [application/octet-stream]
Saving to: ‘cv.csv’

cv.csv              100%[+++++++++++========>]  22.42G  7.69MB/s    in 24m 1s  

2024-10-14 14:16:26 (6.54 MB/s) - ‘cv.csv’ saved [2

In [ ]:
#Если мы хотим работать с полной базой (ОЧЕНЬ ТЯЖЕЛАЯ!)
# df_resum_all = pd.read_csv('/content/cv.csv', on_bad_lines='skip', nrows=100000, sep='|')

In [ ]:
#Подгружаем базу данных (должна передать в уроке),
df_resum = pd.read_excel('/content/samp.xlsx')
#Самостоятельно получить подвыборку в 5000 строк
# df_resum = df_resum_all.sample(5000)

In [ ]:
#Смотрим на три первые строки
df_resum.head(3)


,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,innerInfo
25153,12779aac-d290-11e5-ac86-7de68f1c536d,10e2edb0-d28c-11e5-ac86-7de68f1c536d,2900000000000,2900000100000,"Архангельская-область,-г-Архангельск",1986-03-19T15:00:00+0300,Женский,38.0,кассир,2017-02-21T16:57:52+0300,...,[],"[{""companyName"": ""ОАО АРХРЫБСБЫТ"", ""dateFrom"":...","Сменный-график,Полный-рабочий-день",25000,Полная-занятость,Готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""59e6a6c0-19a3-11e5-8d85-1ff7059456..."
22772,152d0de0-965a-11ec-af0d-5b1b13d7f12f,b6232470-9400-11ec-80f3-a75f74b7b031,2600000000000,2602500001200,"Ставропольский-край,-Труновский-район,-Нижняя-...",1983-01-16T00:00:00+0300,Мужской,41.0,Строитель,2022-02-25T19:43:38+0300,...,[],[],Полный-рабочий-день,40000,Полная-занятость,Готов-к-переобучению,Готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Базовый...",Не-готов-к-переезду,"{""idUser"": ""12c72420-1e3c-11e5-8442-1ff7059456..."
71322,160f6540-8078-11ea-b68f-f3e0740405b3,14e468e0-806f-11ea-b68f-f3e0740405b3,2700000000000,2700000500000,"Хабаровский-край,-г-Комсомольск-на-Амуре",1968-04-25T00:00:00+0300,Мужской,56.0,фотограф,2020-04-17T09:53:04+0300,...,[],[],"Гибкий-график,Полный-рабочий-день",25000,Частичная-занятость,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""1beaa6f0-148f-11e5-8a60-1ff7059456..."


In [ ]:
#Смотрим на информацию
df_resum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   5000 non-null   int64  
 1   id                           5000 non-null   object 
 2   candidateId                  5000 non-null   object 
 3   stateRegionCode              5000 non-null   int64  
 4   locality                     5000 non-null   int64  
 5   localityName                 4982 non-null   object 
 6   birthday                     4858 non-null   object 
 7   gender                       5000 non-null   object 
 8   age                          4858 non-null   float64
 9   positionName                 5000 non-null   object 
 10  dateCreate                   5000 non-null   object 
 11  dateModify                   5000 non-null   object 
 12  publishedDate                4988 non-null   object 
 13  academicDegree    

In [ ]:
df_resum['salary'].max()

999999

In [ ]:
#Отберем нужные данные
col=['localityName', 'birthday', 'age', 'gender', 'positionName',
     'experience',
      'educationList', 'hardSkills', 'softSkills', 'workExperienceList',
      'scheduleType', 'salary', 'busyType', 'languageKnowledge', 'relocation']

In [ ]:
df_resum=df_resum[col]

In [ ]:
df_resum.head()

,localityName,birthday,age,gender,positionName,experience,educationList,hardSkills,softSkills,workExperienceList,scheduleType,salary,busyType,languageKnowledge,relocation
0,"Забайкальский-край,-Тунгокоченский-район,-Верш...",1971-05-29T00:00:00+0300,53.0,Мужской,разнорабочий,0,"[{""type"": ""Education""}]",[],[],"[{""companyName"": ""АО ЗабТЭК"", ""dateFrom"": ""202...",Полный-рабочий-день,35000,Полная-занятость,[],Не-готов-к-переезду
1,Татарстан-республика,1988-11-23T00:00:00+0300,35.0,Мужской,Кладовщик,1,"[{""graduateYear"": 2008, ""instituteName"": ""Проф...",[],[],"[{""companyName"": ""ООО\""АГРОТОРГ\"""", ""dateFrom""...","Сменный-график,Полный-рабочий-день",30000,Полная-занятость,[],Не-готов-к-переезду
2,Башкортостан-республика,NaN,NaN,Мужской,Рабочей,0,"[{""graduateYear"": 1991, ""instituteName"": ""Пту ...",[],[],[],"Гибкий-график,Сменный-график,Полный-рабочий-де...",60000,Полная-занятость,[],Готов-к-переезду
3,"Челябинская-область,-г-Челябинск",1990-12-29T00:00:00+0300,33.0,Женский,Продавец,0,"[{""graduateYear"": 2007, ""instituteName"": ""Школ...",[],[],"[{""companyName"": ""ИП Казаков"", ""dateFrom"": ""20...",Неполный-рабочий-день,25000,Полная-занятость,[],Не-готов-к-переезду
4,Свердловская-область,1973-12-14T00:00:00+0300,50.0,Женский,кладовщик-или-продавец,5,"[{""graduateYear"": 1992, ""instituteName"": ""спту...",[],[],"[{""companyName"": ""ооо \""Пульс\"""", ""dateFrom"": ...",Полный-рабочий-день,30000,Полная-занятость,[],Готов-к-переезду


In [ ]:
# Сбор из таблицы текстовый файл, если данные изначально в таблице
output_text = ""

# Перебор строк DataFrame
for index, row in df_resum.iterrows():
    row_text = ""
    # Мы через "абракадабру" .qqqrrrqqq\n сохраняем связку с номером
    row_text+=f'\n{index}.qqqrrrqqq\n'
    # Проходим по каждой колонка и плюсуем её к тексту резюме с новой строки \n
    for col in df_resum.columns:
        row_text += f"{col}: {row[col]}\n"
    # В конце добавляем ещё одну "абракадабру" чтобы было понятно что резюме закончилось
    row_text += "\nENDOFR\n\n"
    output_text += row_text

# Запись результата в файл
with open('output.txt', 'w') as file:
    file.write(output_text)

print("Текст успешно записан в файл")



Текст успешно записан в файл


In [ ]:
# Смотрим что записалось, выводя первые несколько строк
with open('output.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line, end='')


0.qqqrrrqqq
localityName: Забайкальский-край,-Тунгокоченский-район,-Вершино-Дарасунский-поселок-городского-типа
birthday: 1971-05-29T00:00:00+0300
age: 53.0
gender: Мужской
positionName: разнорабочий
experience: 0
educationList: [{"type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"companyName": "АО ЗабТЭК", "dateFrom": "2022-02-01T00:00:00+0300", "dateTo": "2022-05-01T00:00:00+0300", "demands": "<p>ремонт котельного оборудования</p>", "jobTitle": "слесарь-ремонтник", "relevant": false, "type": "WorkExperience"}]
scheduleType: Полный-рабочий-день
salary: 35000
busyType: Полная-занятость
languageKnowledge: []
relocation: Не-готов-к-переезду

ENDOFR


1.qqqrrrqqq
localityName: Татарстан-республика
birthday: 1988-11-23T00:00:00+0300
age: 35.0
gender: Мужской
positionName: Кладовщик
experience: 1
educationList: [{"graduateYear": 2008, "instituteName": "Профессиональное училище номер 2", "type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: [{"compa

In [ ]:
# def replace_parameters(line):
#     replacements = {
#         "localityName": "НазваниеМестности",
#         "birthday": "ДатаРождения",
#         "age": "Возраст",
#         "gender": "Пол",
#         "positionName": "НазваниеДолжности",
#         "experience": "Опыт",
#         "educationList": "СписокОбразования",
#         "hardSkills": "ЖесткиеНавыки",
#         "softSkills": "МягкиеНавыки",
#         "workExperienceList": "СписокРабочегоОпыта",
#         "companyName": "НазваниеКомпании",
#         "dateFrom": "ДатаНачала",
#         "dateTo": "ДатаОкончания",
#         "demands": "Требования",
#         "jobTitle": "НазваниеРаботы",
#         "relevant": "Актуально",
#         "type": "Тип",
#         "scheduleType": "ТипГрафика",
#         "salary": "Зарплата",
#         "busyType": "ТипЗанятости",
#         "languageKnowledge": "ЗнаниеЯзыков",
#         "relocation": "ГотовностьКПереезду"
#     }

#     for english, russian in replacements.items():
#         line = line.replace(english, russian)
#     return line

# # Чтение файла, замена параметров и запись в новый файл
# with open('output.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()

# with open('output_translated.txt', 'w', encoding='utf-8') as file:
#     for line in lines:
#         file.write(replace_parameters(line))

# print("Файл успешно переведен и сохранен как 'output_translated.txt'")

Файл успешно переведен и сохранен как 'output_translated.txt'


In [ ]:
# Смотрим что записалось, выводя первые несколько строк
# with open('output_translated.txt', 'r', encoding='utf-8') as f:
#     lines = f.readlines()
#     for line in lines[:100]:
#         print(line, end='')

#Создание базы данных FAISS и поисковика

In [ ]:
file_path = '/content/output.txt'
encoding = 'utf8'
delimiter = 'ENDOFR'

# Чтение содержимого файла
with open(file_path, 'r', encoding=encoding) as file:
    text = file.read()

# Разделение текста по заданному разделителю
source_chunks = text.split(delimiter)

# Дополнительно вы можете вывести результат для проверки
print("source_chunks:", source_chunks[2])



source_chunks: 


2.qqqrrrqqq
localityName: Башкортостан-республика
birthday: nan
age: nan
gender: Мужской
positionName: Рабочей
experience: 0
educationList: [{"graduateYear": 1991, "instituteName": "Пту 37 комбинат повышения квалификации электромонтажник", "type": "Education"}]
hardSkills: []
softSkills: []
workExperienceList: []
scheduleType: Гибкий-график,Сменный-график,Полный-рабочий-день,Вахтовый-метод
salary: 60000
busyType: Полная-занятость
languageKnowledge: []
relocation: Готов-к-переезду




In [ ]:
# Преобразование строк в объекты Document (lang_chain)
documents = [Document(page_content=chunk) for chunk in source_chunks]

# Создание базы знаний с использованием FAISS и ранее заданного эмбеддинга
db = FAISS.from_documents(documents, embedding_model)
print(f'База знаний создана!')
#Около 12 минут собирает на cpu

База знаний создана!


In [ ]:
vac='''### Описание вакансии: IT Специалист

#### Основные обязанности:
- Разработка и поддержка программного обеспечения.
- Администрирование и оптимизация работы серверов и сетевых инфраструктур.
- Обеспечение информационной безопасности и защиты данных.

#### Ключевые навыки и требования:

1. **Опыт программирования на Python и JavaScript:**
   - Опыт работы с фреймворками Django или Flask.
   - Глубокое понимание логики объектно-ориентированного программирования.

2. **Знание сетевых технологий и администрирование серверов:**
   - Опыт работы с Linux-серверами (Ubuntu, CentOS).
   - Понимание принципов работы сетевых протоколов (TCP/IP, DNS, HTTP/HTTPS).

3. **Опыт работы с базами данных:**
   - Умение работать с реляционными базами данных (PostgreSQL, MySQL).
   - Знание основ SQL-запросов и оптимизации баз данных.

#### Желательные, но не обязательные требования:
- Опыт работы с облачными платформами (AWS, Google Cloud, Azure).
- Знание контейнеризации и оркестрации (Docker, Kubernetes).
- Опыт работы с системами контроля версий (Git).
- Сертификации в области информационной безопасности (например, CISSP).

#### Кандидатоводительное письмо на наш электронный адрес: hr@company.com. Мы с нетерпением ждем вашего отклика!'''

In [ ]:
key_words=vac
k=5
sim_docs = db.similarity_search(key_words, k=k)

# Функция для извлечения номера строки из текста
def extract_index(text):
    # Используем регулярное выражение для поиска цифр перед ".qqqrrrqqq"
    # Это полезно когда ищещшь не по всей вакансии, а по сущностным моментам!
    match = re.search(r'(\d+)\.qqqrrrqqq', text)
    if match:
        return match.group(1)  # Возвращаем найденное число
    return None  # Если ничего не найдено, возвращаем None




print('Найденные',k, 'фрагментов резюме:\n')

for num, i in enumerate(sim_docs):
    ind=extract_index(i.page_content)
    print('Вакансия:', num+1)
    print('_______________________________________________')
    display(df_resum.loc[int(ind)])
    print()
    print()



Найденные 5 фрагментов резюме:

Вакансия: 1
_______________________________________________


localityName                                 Краснодарский-край,-г-Сочи
birthday                                       1976-02-24T00:00:00+0300
age                                                                48.0
gender                                                          Мужской
positionName             руководитель-подразделения,-ведущий-специалист
experience                                                            3
educationList         [{"faculty": "радиотехнический", "graduateYear...
hardSkills                                                           []
softSkills                                                           []
workExperienceList    [{"achievements": "<p><strong>Профессиональные...
scheduleType          Ненормированный-рабочий-день,Полный-рабочий-де...
salary                                                            80000
busyType                                               Полная-занятость
languageKnowledge     [{"codeLanguage": "Английский", "level": "



Вакансия: 2
_______________________________________________


localityName                            Краснодарский-край,-г-Краснодар
birthday                                       1987-08-04T00:00:00+0400
age                                                                36.0
gender                                                          Мужской
positionName                            Специалист-по-защите-информации
experience                                                            8
educationList         [{"graduateYear": 2009, "instituteName": "Ново...
hardSkills                                                           []
softSkills                                                           []
workExperienceList    [{"companyName": "ООО", "dateFrom": "2015-01-0...
scheduleType                                        Полный-рабочий-день
salary                                                            34999
busyType                                               Полная-занятость
languageKnowledge     [{"codeLanguage": "Английский", "level": "



Вакансия: 3
_______________________________________________


localityName                                           Тульская-область
birthday                                       1988-03-19T00:00:00+0300
age                                                                36.0
gender                                                          Мужской
positionName          Специалист-по-разработке-и-созданию-квантово-о...
experience                                                            0
educationList                                                        []
hardSkills                                                           []
softSkills                                                           []
workExperienceList                                                   []
scheduleType                                        Полный-рабочий-день
salary                                                            45000
busyType                                               Полная-занятость
languageKnowledge                                               



Вакансия: 4
_______________________________________________


localityName                            Костромская-область,-г-Кострома
birthday                                       1993-05-16T16:00:00+0400
age                                                                31.0
gender                                                          Мужской
positionName                                              IT-специалист
experience                                                            6
educationList         [{"faculty": "ФАСТ", "graduateYear": 2015, "in...
hardSkills                                                           []
softSkills                                                           []
workExperienceList    [{"companyName": "Департамент региональной без...
scheduleType                                        Полный-рабочий-день
salary                                                            35000
busyType                                               Полная-занятость
languageKnowledge     [{"codeLanguage": "Английский", "level": "



Вакансия: 5
_______________________________________________


localityName                                                   г-Москва
birthday                                       2000-07-28T16:00:00+0400
age                                                                23.0
gender                                                          Мужской
positionName                                  Менеджер-по-маркетплейсам
experience                                                            0
educationList         [{"faculty": "Информационные системы и програм...
hardSkills                                                           []
softSkills                                                           []
workExperienceList                                                   []
scheduleType                                        Полный-рабочий-день
salary                                                            50000
busyType                                                            NaN
languageKnowledge                                               